# Scraping the French President's Official Agenda 

(L'agenda du Président de la République)

A day by day breakdown of what Macron has been up to since 2017 

Using **BeautifulSoup**

In [26]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup

In [27]:
# the following list is courtesy of chatgpt 

months = ['mai-2017', 'juin-2017', 'juillet-2017', 'aout-2017', 'septembre-2017', 'octobre-2017', 'novembre-2017', 'decembre-2017',
'janvier-2018', 'fevrier-2018', 'mars-2018', 'avril-2018', 'mai-2018', 'juin-2018', 'juillet-2018', 'aout-2018',
'septembre-2018', 'octobre-2018', 'novembre-2018', 'decembre-2018', 'janvier-2019', 'fevrier-2019', 'mars-2019',
'avril-2019', 'mai-2019', 'juin-2019', 'juillet-2019', 'aout-2019', 'septembre-2019', 'octobre-2019', 'novembre-2019',
'decembre-2019', 'janvier-2020', 'fevrier-2020', 'mars-2020', 'avril-2020', 'mai-2020', 'juin-2020', 'juillet-2020',
'aout-2020', 'septembre-2020', 'octobre-2020', 'novembre-2020', 'decembre-2020', 'janvier-2021', 'fevrier-2021',
'mars-2021', 'avril-2021', 'mai-2021', 'juin-2021', 'juillet-2021', 'aout-2021', 'septembre-2021', 'octobre-2021',
'novembre-2021', 'decembre-2021', 'janvier-2022', 'fevrier-2022', 'mars-2022', 'avril-2022', 'mai-2022', 'juin-2022',
'juillet-2022', 'aout-2022', 'septembre-2022', 'octobre-2022', 'novembre-2022', 'decembre-2022', 'janvier-2023',
'fevrier-2023', 'mars-2023']


In [28]:
# this part I wrote - it goes through the agenda for each available month and
all_data = []

for month in months:
    url = f'https://www.elysee.fr/agenda-{month}'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    sections = soup.find_all('section', {'class': 'container space-bottom-s'})

    for section in sections:
            date = section.find('span', {'class': 'text-up'}).text
            events = section.find_all('div', {'class': "list-table__content"})
            for event in events:
                item = {}
                time = event.find('span', {'class': 'list-table__hour'}).text
                desc = event.find('div', {'class': 'm-b-n'}).text
                try:
                    type = event.find('span', {'class': 'list-table__type'}).text
                except:
                    type = None
                item["date"] = date
                item["month"] = month
                item["time"] = time
                item["desc"] = desc
                item["type"] = type
                all_data.append(item)


In [29]:
all_data

[{'date': '14',
  'month': 'mai-2017',
  'time': '10h00',
  'desc': "Passation de pouvoir et cérémonie d'investiture",
  'type': None},
 {'date': '14',
  'month': 'mai-2017',
  'time': '12h00',
  'desc': "Cérémonie à l'hôtel de Ville de Paris",
  'type': None},
 {'date': '17',
  'month': 'mai-2017',
  'time': '20h00',
  'desc': 'Dîner avec M. Donald Tusk, Président du Conseil européen',
  'type': None},
 {'date': '18',
  'month': 'mai-2017',
  'time': '11h00',
  'desc': 'Conseil des ministres',
  'type': None},
 {'date': '18',
  'month': 'mai-2017',
  'time': '14h30',
  'desc': 'Conseil de défense',
  'type': None},
 {'date': '21',
  'month': 'mai-2017',
  'time': '20h00',
  'desc': 'Réception de M. Paolo GENTILONI, Président du Conseil des ministres de la République italienne',
  'type': None},
 {'date': '23',
  'month': 'mai-2017',
  'time': '09h00',
  'desc': 'Entretien avec M. Laurent BERGER, Secrétaire général de la CFDT',
  'type': None},
 {'date': '23',
  'month': 'mai-2017',
  

In [30]:
with open('elysee.json', 'w') as f:
    json.dump(all_data, f, indent=4)

In [31]:
df = pd.DataFrame.from_dict(all_data)

In [32]:
df

,date,month,time,desc,type
0,14,mai-2017,10h00,Passation de pouvoir et cérémonie d'investiture,None
1,14,mai-2017,12h00,Cérémonie à l'hôtel de Ville de Paris,None
2,17,mai-2017,20h00,"Dîner avec M. Donald Tusk, Président du Consei...",None
3,18,mai-2017,11h00,Conseil des ministres,None
4,18,mai-2017,14h30,Conseil de défense,None
...,...,...,...,...,...
3715,16,mars-2023,14h40,Conseil des ministres.,Conseil des ministres
3716,16,mars-2023,16h00,"Entretien M. Sultan AL JABER, Président de la ...",Entretien
3717,17,mars-2023,10h00,"Entretien téléphonique avec Mme. Maia SANDU, P...",Entretien
3718,17,mars-2023,10h30,Réception des lauréats de la deuxième promotio...,Réception


In [33]:
# split the month column into year and month 

df['year'] = df['month'].str.split('-').str[1]
df['month'] = df['month'].str.split('-').str[0]

In [34]:
df

,date,month,time,desc,type,year
0,14,mai,10h00,Passation de pouvoir et cérémonie d'investiture,None,2017
1,14,mai,12h00,Cérémonie à l'hôtel de Ville de Paris,None,2017
2,17,mai,20h00,"Dîner avec M. Donald Tusk, Président du Consei...",None,2017
3,18,mai,11h00,Conseil des ministres,None,2017
4,18,mai,14h30,Conseil de défense,None,2017
...,...,...,...,...,...,...
3715,16,mars,14h40,Conseil des ministres.,Conseil des ministres,2023
3716,16,mars,16h00,"Entretien M. Sultan AL JABER, Président de la ...",Entretien,2023
3717,17,mars,10h00,"Entretien téléphonique avec Mme. Maia SANDU, P...",Entretien,2023
3718,17,mars,10h30,Réception des lauréats de la deuxième promotio...,Réception,2023


In [35]:
df_with_type = df[df['type'].notna()]

df_with_type

,date,month,time,desc,type,year
1102,03,octobre,10h00,Conseil des ministres.,Conseil des ministres,2018
1139,24,octobre,10h00,Conseil des ministres.,Conseil des ministres,2018
1142,25,octobre,15h00,Entretien avec avec Mme Nadia MURAD BASEE TAHA...,Rencontre,2018
1159,06,novembre,10h00,Déplacement aux Eparges. Meuse,Discours,2018
1161,06,novembre,18h00,Déplacement à Reims. Marne,Discours,2018
...,...,...,...,...,...,...
3715,16,mars,14h40,Conseil des ministres.,Conseil des ministres,2023
3716,16,mars,16h00,"Entretien M. Sultan AL JABER, Président de la ...",Entretien,2023
3717,17,mars,10h00,"Entretien téléphonique avec Mme. Maia SANDU, P...",Entretien,2023
3718,17,mars,10h30,Réception des lauréats de la deuxième promotio...,Réception,2023


In [36]:
# count the number of rows by type 

df_with_type['type'].value_counts()

Entretien                864
Déjeuner                 241
Conseil des ministres    212
Conseil de défense       210
Cérémonie officielle     155
Voyage officiel          149
Déplacement              140
Sommet                   127
Rencontre                117
Réunion                   81
Discours                  77
Réception                 72
Dîner                     60
Remise de rapport         21
Inauguration               9
Conférence                 8
Conférence de presse       8
Interview                  7
Vote                       1
Visite d'État              1
Name: type, dtype: int64

In [37]:
# save to csv

df_with_type.to_csv('elysee_types.csv', index=False)

In [38]:
df.head()

,date,month,time,desc,type,year
0,14,mai,10h00,Passation de pouvoir et cérémonie d'investiture,None,2017
1,14,mai,12h00,Cérémonie à l'hôtel de Ville de Paris,None,2017
2,17,mai,20h00,"Dîner avec M. Donald Tusk, Président du Consei...",None,2017
3,18,mai,11h00,Conseil des ministres,None,2017
4,18,mai,14h30,Conseil de défense,None,2017


In [39]:
# replace the months with numbers

df['month'] = df['month'].replace({'janvier': '01', 'fevrier': '02', 'mars': '03', 'avril': '04', 'mai': '05', 'juin': '06', 'juillet': '07', 'aout': '08', 'septembre': '09', 'octobre': '10', 'novembre': '11', 'decembre': '12'})

In [40]:
df

,date,month,time,desc,type,year
0,14,05,10h00,Passation de pouvoir et cérémonie d'investiture,None,2017
1,14,05,12h00,Cérémonie à l'hôtel de Ville de Paris,None,2017
2,17,05,20h00,"Dîner avec M. Donald Tusk, Président du Consei...",None,2017
3,18,05,11h00,Conseil des ministres,None,2017
4,18,05,14h30,Conseil de défense,None,2017
...,...,...,...,...,...,...
3715,16,03,14h40,Conseil des ministres.,Conseil des ministres,2023
3716,16,03,16h00,"Entretien M. Sultan AL JABER, Président de la ...",Entretien,2023
3717,17,03,10h00,"Entretien téléphonique avec Mme. Maia SANDU, P...",Entretien,2023
3718,17,03,10h30,Réception des lauréats de la deuxième promotio...,Réception,2023


In [41]:
# make a new column with the date in the format YYYY-MM-DD

df['date'] = df['year'] + '-' + df['month'] + '-' + df['date']


In [42]:
# make a new column called event where it is true 

df['event'] = True

In [43]:
df

,date,month,time,desc,type,year,event
0,2017-05-14,05,10h00,Passation de pouvoir et cérémonie d'investiture,None,2017,True
1,2017-05-14,05,12h00,Cérémonie à l'hôtel de Ville de Paris,None,2017,True
2,2017-05-17,05,20h00,"Dîner avec M. Donald Tusk, Président du Consei...",None,2017,True
3,2017-05-18,05,11h00,Conseil des ministres,None,2017,True
4,2017-05-18,05,14h30,Conseil de défense,None,2017,True
...,...,...,...,...,...,...,...
3715,2023-03-16,03,14h40,Conseil des ministres.,Conseil des ministres,2023,True
3716,2023-03-16,03,16h00,"Entretien M. Sultan AL JABER, Président de la ...",Entretien,2023,True
3717,2023-03-17,03,10h00,"Entretien téléphonique avec Mme. Maia SANDU, P...",Entretien,2023,True
3718,2023-03-17,03,10h30,Réception des lauréats de la deuxième promotio...,Réception,2023,True


In [47]:
# get the data from 2018

df_2018 = df[df['year'] == '2018']


In [48]:
# save it to csv

df_2018.to_csv('elysee_2018.csv', index=False)

In [44]:
df_2022 = df[df['year'] >= '2022']

In [51]:
df_2022["type"].value_counts()

Entretien                298
Conseil des ministres     57
Déjeuner                  53
Déplacement               52
Sommet                    41
Cérémonie officielle      39
Conseil de défense        35
Réunion                   35
Voyage officiel           25
Dîner                     23
Réception                 18
Discours                  18
Rencontre                 13
Conférence                 7
Remise de rapport          6
Conférence de presse       3
Inauguration               3
Interview                  2
Vote                       1
Visite d'État              1
Name: type, dtype: int64

In [52]:
# make a new True / False column for 2022 where it is true if the event type is Déjeuner

df_2022['dejeuner'] = df_2022['type'] == 'Déjeuner'

df_2022['entretien'] = df_2022['type'] == 'Entretien'

df_2022['deplacement'] = df_2022['type'] == 'Déplacement'

df_2022['voyage'] = df_2022['type'] == 'Voyage officiel'

/var/folders/_0/x7xh_2c16jq4chdcqrg28fr40000gn/T/ipykernel_76882/2689066865.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['dejeuner'] = df_2022['type'] == 'Déjeuner'
/var/folders/_0/x7xh_2c16jq4chdcqrg28fr40000gn/T/ipykernel_76882/2689066865.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['entretien'] = df_2022['type'] == 'Entretien'
/var/folders/_0/x7xh_2c16jq4chdcqrg28fr40000gn/T/ipykernel_76882/2689066865.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [53]:
df_2022

,date,month,time,desc,type,year,event,dejeuner,entretien,deplacement,voyage
2990,2022-01-03,01,13h15,"Déjeuner avec M. Jean CASTEX, Premier ministre.",Déjeuner,2022,True,True,False,False,False
2991,2022-01-04,01,18h00,Déplacement au Conseil d’État.,Déplacement,2022,True,False,False,True,False
2992,2022-01-05,01,09h30,Conseil de défense COVID-19.,Conseil de défense,2022,True,False,False,False,False
2993,2022-01-05,01,10h30,Conseil des ministres.,Conseil des ministres,2022,True,False,False,False,False
2994,2022-01-05,01,18h00,Rencontre avec les autorités religieuses.,Rencontre,2022,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
3715,2023-03-16,03,14h40,Conseil des ministres.,Conseil des ministres,2023,True,False,False,False,False
3716,2023-03-16,03,16h00,"Entretien M. Sultan AL JABER, Président de la ...",Entretien,2023,True,False,True,False,False
3717,2023-03-17,03,10h00,"Entretien téléphonique avec Mme. Maia SANDU, P...",Entretien,2023,True,False,True,False,False
3718,2023-03-17,03,10h30,Réception des lauréats de la deuxième promotio...,Réception,2023,True,False,False,False,False


In [45]:
# save to csv

df_2022.to_csv('elysee_2022.csv', index=False)

In [46]:
# save df to csv

df.to_csv('elysee.csv', index=False)